In [11]:
import rlepose.datasets.lxd_freihand as lxd_freihand
import rlepose.datasets.mytransform as mytransform
root_dir = "/home/louxd/dataset/FreiHand"
split0_train = "FreiHAND_pub_v2/training"
split1_train = "FreiHAND_pub_v2"
split2_train = "training"
split0_eval = "FreiHAND_pub_v2_eval/evaluation"
split1_eval = "FreiHAND_pub_v2_eval"
split2_eval = "evaluation"
mode_train = "train"
mode_eval = "eval"
batch_size = 32

train_transforms = mytransform.Compose([
                                    # Mytransforms.RandomResized(),
                                    # Mytransforms.RandomRotate(40),
                                    # Mytransforms.RandomCrop(368),
                                    # Mytransforms.RandomHorizontalFlip(),
                                    mytransform.TestResized(368),
                                    ])

eval_transforms = mytransform.Compose([
                                        mytransform.TestResized(368),
                                        ])

train_dataset = lxd_freihand.FreiHand_RLE(root_dir, split0_train, split1_train, split2_train, 
                                            mode=mode_train, stride=8, transformer=train_transforms)
# modi end

In [13]:
import random
import numpy as np
import torch
import torch.multiprocessing as mp
import torch.utils.data

num_gpu = torch.cuda.device_count()


def _init_fn(worker_id):
    np.random.seed(1234)
    random.seed(1234)

train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=16, rank=5)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None), num_workers=16, sampler=train_sampler, worker_init_fn=_init_fn)
